# Text adventure game

This Python notebook builds a simple text advenutre game inspired by the [Adventuron Classroom](https://adventuron.io/classroom/) design by Chris Ainsley of Adventuron Software Limited.

There are two main components to the game:
1. __The parser__ interprets the player's commands.
2. __The game__ represents the world (a collection of locations and items), and describes what the player sees.

In [0]:
class Game:
  """The Game class represents the world.  Internally, we use a 
     graph of Location objects and Item objects, which can be at a 
     Location or in the player's inventory.  Each locations has a set of
     exits which are the directions that a player can move to get to an
     adjacent location. The player can move from one location to another
     location by typing a command like "Go North".
  """

  def __init__(self, start_at):
    # start_at is the location in the game where the player starts
    self.curr_location = start_at
    # inventory is the set of objects that the player has collected/
    self.inventory = {}
    # Booleans is used to keep track of whether certain puzzles are solved.
    # It includes things like whether an exit is blocked.
    self.booleans = {}

  def describe(self):
    """Describe the current game state by first describing the current 
       location, then listing any exits, and then describing any objects
       in the current location."""
    self.describe_current_location()
    self.describe_exits()
    self.describe_items()

  def describe_current_location(self):
    """Describe the current location by printing its description field."""
    print(self.curr_location.description)

  def describe_exits(self):
    """List the directions that the player can take to exit from the current
       location."""
    exits = []
    for exit in self.curr_location.connections.keys():
      exits.append(exit.capitalize())
    if len(exits) > 0:
      print("Exits: ", end = '')
      print(*exits, sep = ", ",)
  
  def describe_items(self):
    """Describe what objects are in the current location."""
    if len(self.curr_location.items) > 0:
      print("You see: ")
      for item_name in self.curr_location.items:
        item = self.curr_location.items[item_name]
        print(item.description)

  def set_boolean(self, bool_name, bool_value):
    """Create or update a boolean value.  Booleans are used for 
       tracking puzzle solutions."""
    self.booleans[bool_name] = bool_value
  
  def get_boolean(self, bool_name):
    """Get a boolean value by name."""
    if not bool_name in self.booleans:
      return False
    else:
      return self.booleans[bool_name]


class Location:
  """Locations are the places in the game that a player can visit.
     Internally they are represented nodes in a graph.  Each location stores
     a description of the location, any items in the location, its connections
     to adjacent locations, and any blocks that prevent movement to an adjacent
     location.  The connections are  arcs between locations that are labeled 
     with the direction that the player can use to go to the adjacent location. 
  """
  def __init__(self, description):
    self.description = description
    self.connections = {}
    self.items = {}
    self.blocks = {}

  def add_connection(self, direction, connected_location):
    """Add a connection from the current location to a connected location.
       Direction is a string that the player can use to get to the connected
       location.  If the direction is a cardinal direction, then we also 
       automatically make a connection in the reverse direction."""
    self.connections[direction] = connected_location
    if direction == 'north':
      connected_location.connections["south"] = self
    if direction == 'south':
      connected_location.connections["north"] = self
    if direction == 'east':
      connected_location.connections["west"] = self
    if direction == 'west':
      connected_location.connections["east"] = self
    if direction == 'up':
      connected_location.connections["down"] = self
    if direction == 'down':
      connected_location.connections["up"] = self


  def is_blocked(self, direction):
    """Check to see whether its possible to move in the direction, or if it
       is blocked."""
    if not self.connections[direction]:
      return True
    else:
      if not direction in self.blocks:
        return False
      block = self.blocks[direction]
      return block.is_blocked()


  def add_item(self, name, item):
    """Put an item in this location."""
    self.items[name] = item

  def remove_item(self, item):
    """Remove an item from this location (for instance, if the player picks it
       up and puts it in their inventory)."""
    self.items.pop(item.name)

  def add_block(self, blocked_location, block):
    """Create an obstacle that prevents a player from moving to the blocked 
       location."""
    blocked_direction = None
    for direction in self.connections:
      if self.connections[direction] == blocked_location:
         blocked_direction = direction
    if blocked_direction:
      self.blocks[blocked_direction] = block

class Item:
  """Items are objects that a player can get, or scenery that a player can
     examine."""
  def __init__(self, name, description, examine_text="", start_at=None, gettable=True):
    # The name of the object
    self.name = name
    # The default description of the object.
    self.description = description
    # The detailed description of the player examines the object.
    self.examine_text = examine_text
    # Indicates whether a player can get the object and put it in their inventory.
    self.gettable = gettable
    # The location in the Game where the object starts.
    if start_at:
      start_at.add_item(name, self)
    self.upon_examine = self.do_nothing

  def set_function_to_use_upon_examine(self, function):
    """You can override the function that gets called when a player examines
       the object. The default behavior is to just describe the Item and do 
       nothing else.  However, sometimes we want the act of examining an object
       to create new objects.  For example, "Examine rose bush" might reveal a 
       hidden key, which could be created via this function. 
    """
    self.upon_examine = function

  def do_nothing(self):
    """ This is the default function that is called when an item is examined."""
    return False


class Block:
  """A Block is an obstacle that prevents a player from moving to a location."""  
  def __init__(self, location, description, check):
    # is the block currently active, or has it been solved?
    self.is_active = True
    # the blocked location
    self.location = location
    # A descrption that tells the player why they can't move to the location
    self.description = description
    # The name (String) of the boolean in the Game to check whether the block
    # has been removed or not.
    self.check = check

  
  def is_blocked(self):
    """Checks whether they player has removed the block by solving a puzzle
       (stored in the Game's boolean set)."""
    if game.get_boolean(self.check):
       self.is_active = False
    else:
      self.is_active = True 
    
    return self.is_active


  


In [0]:
class Parser:
  """The Parser is the class that handles the player's input.  The player 
     writes commands, and the parser performs natural language understanding
     in order to interpret what the player intended, and how that intent
     is reflected in the simulated world. 
  """
  def __init__(self, game):
    # A list of all of the commands that the player has issued.
    self.command_history = []
    # A pointer to the game.
    self.game = game

  def parse_command(self, command):
    
    # add this command to the history
    self.command_history.append(command)

    # process direction commands
    direction = self.get_direction(command)
    if direction:
      if direction in self.game.curr_location.connections:
        if self.game.curr_location.is_blocked(direction):
          # check to see whether that direction is blocked.
          block = self.game.curr_location.blocks[direction]
          print(block.description)
        else:
          # if it's not blocked, then move there 
          self.game.curr_location = self.game.curr_location.connections[direction]
          self.game.describe()
      else:
        print("You can't go %s from here." % direction.capitalize())

    # when the user issues a "look" command, re-describe what they see
    elif command.lower() == "look" or command.lower() == "l":
      self.game.describe()

    # process "examine" commands
    elif "examine " in command or command.lower().startswith("x "):
      self.examine(command)
    
    # process "take" commands
    elif "take " in command or "get " in command:
      self.take(command)

    # process "drop" commands
    elif "drop " in command:
      self.drop(command)

    # process "inventory" command
    elif "inventory" in command or command.lower() == "i":
      if len(self.game.inventory) == 0:
        print("You don't have anything.")
      else:
        descriptions = []
        for item_name in self.game.inventory:
          item = self.game.inventory[item_name]
          descriptions.append(item.description)
        print("You have: ", end = '')
        print(*descriptions, sep = ", ",)
    
    # special commands for this game
    elif "stand ladder" in command.lower() or "lean ladder" in command.lower() or "raise ladder" in command.lower():
      if 'ladder' in self.game.inventory:
        if self.game.curr_location == foot_of_tree:
          item = self.game.inventory.pop('ladder')
          self.game.curr_location.add_item('ladder', item)
          print("You raise the ladder.")
          game.set_boolean("is_ladder_standing", True)
          self.game.describe()
      else:
        print("You don't have a ladder.")

  def examine(self, command):
    command = command.lower()
    matched_item = False
    # check whether any of the items at this location match the command
    for item_name in self.game.curr_location.items:
      if item_name in command:
        item = self.game.curr_location.items[item_name]
        if item.examine_text:
          print(item.examine_text)
          matched_item = True
        if item.upon_examine():
          matched_item = True
        break
    # check whether any of the items in the inventory match the command
    for item_name in self.game.inventory:
      if item_name in command:
        item = self.game.inventory[item_name]
        if item.examine_text:
          print(item.examine_text)
          matched_item = True
    # fail
    if not matched_item:
      print("You don't see anything special.")


  def take(self, command):
    command = command.lower()
    matched_item = False
    # check whether any of the items at this location match the command
    for item_name in self.game.curr_location.items:
      if item_name in command:
        item = self.game.curr_location.items[item_name]
        if item.gettable:
          self.game.inventory[item_name] = item
          self.game.curr_location.remove_item(item)
          print("You take the %s." % item_name)
        else:
          print("You cannot the %s." % item_name)
        matched_item = True
        break
    # check whether any of the items in the inventory match the command
    if not matched_item:
      for item_name in self.inventory:
        if item_name in command:
          print("You already have the %s." % item_name)
          matched_item = True
    # fail
    if not matched_item:
      print("You can't find it.")

  def drop(self, command):
    command = command.lower()
    matched_item = False
    # check whether any of the items in the inventory match the command
    if not matched_item:
      for item_name in self.inventory:
        if item_name in command:
          matched_item = True
          item = self.inventory[item_name]
          self.game.curr_location.add_item(item_name, item)
          self.game.inventory.pop(item_name)
          print("You drop the %s." % item_name)
          break
    # fail
    if not matched_item:
      print("You don't have that.")

  def get_direction(self, command):
    command = command.lower()
    if command == "n" or "north" in command:
      return "north" 
    if command == "s" or "south" in command:
      return "south"
    if command == "e" or "east" in command: 
      return "east"
    if command == "w" or "west" in command:
      return "west"
    if command == "up":
      return "up"
    if command == "down":
      return "down"
    return None

In [0]:
# Locations
hut = Location("You are in an old hut. Sunlight shines in from a doorway to the north.")
road_1 = Location("You are standing on a track. The hut is south.")
road_2 = Location("You are where the road turns eastwards. You see small hills in the distance.")
road_3 = Location("You are on a road surrounded by grass. Trees can be seen on a hill in the distance.")
road_4 = Location("You are on a dusty path on the edge of Birwood.")
lamp_seller_on_road = Location("You are at a crossroads.")
chasm = Location("You are stood on the edge of a huge chasm (a cliff). A tightrope spans the gap, but it looks dangerous.")
edge_of_birwood = Location("You are on the edge of Birwood. A rope spans the chasm northwards." )
deep_inside_birwood = Location("You are deep inside Birwood. The trees are alive with the buzzing of tiny insects.")
birwood_bush = Location("You are standing in Birwood. A gap in the trees lets a warm light filter through.")
demon_knight_road = Location("You are in a dip in the road by dark Birwood. Birds can be heard above.")
foot_of_tree = Location("You are at the bottom of a large stone tree that is bare of any branches. The road ends here.")
top_of_tree = Location("You are in a stone room set in a large stone tree.")
castle_approach_1 = Location("You are now a fair distance from Birwood. You can see castle Camelot to the east.")
castle_approach_2 = Location("You are outside Camelot Castle. A great stone door is the only way in.")
castle_porch = Location("You are in the porch of Camelot.")
banquet_hall = Location("You are in an abandoned banquet hall (eating area). Furniture lays broken on the floor.")
drafty_room = Location("You are in a drafty room. Wind blows through gaps in the walls creating howling noises.")
ornate_antechamber = Location("You are in an antechamber covered in thin ice that even covers the paintings.")
portcullis = Location("You are in a room which has an iron portcullis set into the northern wall.")
salt_mine_1 = Location("You are in an old salt mine. Tunnels lead to the west and south.")
salt_mine_2 = Location("You are in the west part of the mine. It looks like it's been abandoned.")
salt_mine_3 = Location("You are at the end of the mine. You can hear the drip of water.")
worm_room = Location("You are in the south part of the mine. There been recent movement in the rocks near your feet.")
cold_room = Location("You are in a bitterly cold room. Everything is coated in a thick layer of ice.")
winch_room = Location("You are in the winch room.")
armoury = Location("You are in the armoury. Empty weapon racks line the walls.")
most_splendid_room = Location("You are in the most splendid room in the castle. Rugs and paintings adorn the floor and walls.")
arthur = Location("You are in a sparse and lonely room. A cold wind enters through a high window.")

# Connections
hut.add_connection("north", road_1)
road_1.add_connection("north", road_2)
road_2.add_connection("east", road_3)
road_3.add_connection("east", road_4)
road_4.add_connection("north", foot_of_tree)
road_4.add_connection("south", lamp_seller_on_road)
lamp_seller_on_road.add_connection("east", demon_knight_road)
lamp_seller_on_road.add_connection("south", chasm)
foot_of_tree.add_connection("up",  top_of_tree)
chasm.add_connection("south", edge_of_birwood)
edge_of_birwood.add_connection("south", deep_inside_birwood)
deep_inside_birwood.add_connection("west", birwood_bush)
demon_knight_road.add_connection("east", castle_approach_1)
castle_approach_1.add_connection("east", castle_approach_2)
castle_approach_2.add_connection("east", castle_porch)
castle_porch.add_connection("east", banquet_hall)
banquet_hall.add_connection("north", portcullis)
banquet_hall.add_connection("south", drafty_room)
portcullis.add_connection("north", most_splendid_room)
most_splendid_room.add_connection("west", arthur)
drafty_room.add_connection("east", ornate_antechamber)
drafty_room.add_connection("down", salt_mine_1)
ornate_antechamber.add_connection("east", cold_room)
cold_room.add_connection("north", winch_room)
winch_room.add_connection("east", armoury)
salt_mine_1.add_connection("west", salt_mine_2)
salt_mine_1.add_connection("south", worm_room)
worm_room.add_connection("south", salt_mine_3)

# Items that you can pick up
#ladder = Item("ladder", "a ladder", "A LONG POLE WITH RUNGS ATTACHED.", start_at=hut)
ladder = Item("ladder", "a ladder", "A LONG POLE WITH RUNGS ATTACHED.", start_at=foot_of_tree)
red_fish = Item("fish", "a red fish", "IT STINKS!", start_at=road_3)
short_sword = Item("sword", "a short sword", "INSCRIBED UPON IT ARE THE WORDS 'GOOD LUCK' ~~ MERLIN.", start_at=top_of_tree)
coin = Item("coin", "a coin", start_at=ornate_antechamber)
string = Item("string", "some string", start_at=salt_mine_2)
salt = Item("salt", "some salt", start_at=salt_mine_3)
oil = Item("oil", "can of oil", start_at=cold_room)
excalibur = Item("excalibur", "Excalibur!", start_at=armoury)
long_pole = Item("pole", "a long pole")
rungs = Item("rungs", "some rungs")
stone_key = Item("key", "a stone key")
axe = Item("axe", "a wood cutters' axe")
lamp = Item("lamp", "a lamp", "IT'S OLD AND TARNISHED.")

# Things or people that we cannot pick up (Scenery)
old_woman= Item("woman", "an old woman selling lamps", "SHE LOOKS AT YOU WITH AN INTENSE GLARE.", start_at=lamp_seller_on_road, gettable=False)
#old_woman= Item("woman", "an old woman selling lamps", "SHE LOOKS AT YOU WITH AN INTENSE GLARE.", start_at=hut, gettable=False)
logs= Item("logs", "a pile of logs", start_at=deep_inside_birwood, gettable=False)
bush= Item("bush", "a bush", start_at=birwood_bush, gettable=False)
demon_knight= Item("knight", "a Demon Knight guarding the east road", start_at=demon_knight_road, gettable=False)
trapdoor= Item("trapdoor", "an old trapdoor", start_at=drafty_room, gettable=False)
smashed_trapdoor= Item("trapdoor", "a smashed trapdoor", gettable=False)
rockworm= Item("rockworm", "a Rockworm, guarding the south tunnel", start_at=worm_room, gettable=False)
ice_creature= Item("ice creature", "an Ice creature, guarding the north exit", start_at=cold_room, gettable=False)
winch= Item("winch", "a winch", start_at=winch_room, gettable=False)
spell= Item("spell", "a spell flying at you from Crania", start_at=most_splendid_room, gettable=False)
asleep_arthur= Item("king arthur", "King Arthur (asleep)", start_at=arthur, gettable=False)


# Barriers/Blocks
tree_block = Block(top_of_tree, "YOU CAN'T FIND YOUR GRIP", "is_ladder_standing")

foot_of_tree.add_block(top_of_tree, tree_block)


# Special functions
def examine_bush():
  print("You found a stone key!")
  game.curr_location.items[stone_key.name] = stone_key
  game.describe()
  description_was_given = True
  # Revert to the regular behavior after this has been run once.
  bush.set_function_to_use_upon_examine(bush.do_nothing)
  return description_was_given
# Attach the special function to the object
bush.set_function_to_use_upon_examine(examine_bush)

def examine_logs():
  print("You found an axe!")
  game.curr_location.items[axe.name] = axe
  game.describe()
  description_was_given = True
  # Revert to the regular behavior after this has been run once.
  logs.set_function_to_use_upon_examine(logs.do_nothing)
  return description_was_given
logs.set_function_to_use_upon_examine(examine_logs)




In [5]:
#game = Game(hut)
game = Game(foot_of_tree)
game.set_boolean("is_ladder_standing", False)


parser = Parser(game)
game.describe()
command = ""
while not (command.lower() == "exit"):
  command = input(">")
  parser.parse_command(command)


You are at the bottom of a large stone tree that is bare of any branches. The road ends here.
Exits: South, Up
You see: 
a ladder
>raise ladder
You don't have a ladder.
>take ladder
You take the ladder.
>raise ladder
You raise the ladder.
You are at the bottom of a large stone tree that is bare of any branches. The road ends here.
Exits: South, Up
You see: 
a ladder
>up
You are in a stone room set in a large stone tree.
Exits: Down
You see: 
a short sword
>take sword
You take the sword.
>examine sword
INSCRIBED UPON IT ARE THE WORDS 'GOOD LUCK' ~~ MERLIN.
>exut
>exit


In [6]:
parser.command_history


['raise ladder',
 'take ladder',
 'raise ladder',
 'up',
 'take sword',
 'examine sword',
 'exut',
 'exit']